<a href="https://colab.research.google.com/github/Noob919/Numpy_excerise/blob/main/Sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing all the necessary libraries

In [1]:
import numpy as np
import pandas as pd

In [34]:
path_train = "/content/drive/MyDrive/kaggle/yelp_review_polarity_csv/train.csv"
path_test = "/content/drive/MyDrive/kaggle/yelp_review_polarity_csv/train.csv"

In [35]:
train_df  = pd.read_csv(path_train)

In [36]:
test_df = pd.read_csv(path_test)

In [37]:
train_df.columns = ['Rating', 'Reviews']
test_df.columns = ['Rating', 'Reviews']

In [38]:
train_df['Rating'] = train_df['Rating'].replace(1,0)
test_df['Rating'] = test_df['Rating'].replace(1,0)

In [39]:
train_df['Rating'] = train_df['Rating'].replace(2,1)
test_df['Rating'] = test_df['Rating'].replace(2,1)

In [40]:
pip install transformers

In [41]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels  = 2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [43]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [44]:
train_df = train_df[0:10000]
test_df  = test_df[0:3000]

In [45]:
train_df.shape

(10000, 2)

In [13]:
import tensorflow as tf

In [46]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, test_df, 'Reviews', 'Rating')

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [48]:
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics= ['accuracy'])

In [49]:
model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
626/626 [==============================] - 977s 2s/step - loss: 0.7335 - accuracy: 0.5282 - val_loss: 0.6931 - val_accuracy: 0.5387
Epoch 2/2
626/626 [==============================] - 954s 2s/step - loss: 0.6931 - accuracy: 0.5293 - val_loss: 0.6931 - val_accuracy: 0.5387


In [50]:
pd.DataFrame(model.history.history)

,loss,accuracy,val_loss,val_accuracy
0,0.733508,0.5282,0.693147,0.538667
1,0.693146,0.5293,0.693147,0.538667
